In [1]:
import docplex.mp, numpy as np, matplotlib.pyplot as plt
from docplex.mp.model import Model
from scipy.stats import norm

#### Uncertain demands
$$
	{\min}_x \ 2x_1+3x_2\\ 
s.t.\ \ 	x_1 + x_2 \le 100 \\
\ \ 	2x_1 + 6x_2 \ge h_1(\xi) \\
\ \ 	3x_1 + 3x_2 \ge h_2(\xi) \\
\ \ 	x_1,x_2\ge 0 \\
$$

$h_1(\xi)=180+\zeta_1, \zeta_1$ ~ $\mathcal{N}(0,16)$

$h_2(\xi)=163+\zeta_2, \zeta_2$ ~ $\mathcal{N}(0,9)$

1. deterministic
2. worst-case
3. scenario analysis
4. chance-constrained
5. recourse model
6. evaluating model performances

In [2]:
num_prod=2
c=[2,3]
m=2 #dim of uncertainty
capacity=100
prod_rate=[[2,6],[3,3]]
demands=[180,163]

### 1. deterministic

In [3]:
md1 = Model(name='deterministic')
x = {(i): md1.continuous_var(name='x_{0}'.format(i)) for i in range(num_prod)}

In [4]:
for i in range(m):
    md1.add_constraint(md1.sum(x[j]*prod_rate[i][j] for j in range(num_prod)) >= demands[i])
md1.add_constraint(md1.sum(x[j] for j in range(num_prod))<=capacity)
obj1=md1.minimize(md1.sum(x[i]*c[i] for i in range(num_prod)))
md1.print_information()

Model: deterministic
 - number of variables: 2
   - binary=0, integer=0, continuous=2
 - number of constraints: 3
   - linear=3
 - parameters: defaults


In [5]:
md1.solve().display()

solution for: deterministic
objective: 126.500
x_0 = 36.500
x_1 = 17.833


### 2. worst-case

In [6]:
h1_99=norm.ppf(0.99,loc=0,scale=16)
h2_99=norm.ppf(0.99,loc=0,scale=9)
h=[h1_99,h2_99]

In [7]:
md2 = Model(name='worst_case')
x = {(i): md2.continuous_var(name='x_{0}'.format(i)) for i in range(num_prod)}

for i in range(m):
    md2.add_constraint(md2.sum(x[j]*prod_rate[i][j] for j in range(num_prod)) >= demands[i]+h[i])
md2.add_constraint(md2.sum(x[j] for j in range(num_prod))<=capacity)
obj2=md2.minimize(md2.sum(x[i]*c[i] for i in range(num_prod)))
md2.print_information()

Model: worst_case
 - number of variables: 2
   - binary=0, integer=0, continuous=2
 - number of constraints: 3
   - linear=3
 - parameters: defaults


In [8]:
md2.solve().display()

solution for: worst_case
objective: 146.274
x_0 = 37.663
x_1 = 23.649


### 3. scenario analysis

In [9]:
K=500
h_scenario=[(np.random.normal(loc=0,scale=16),np.random.normal(loc=0,scale=9)) for k in range(K)]
scenario_perf=[]
scenario_x=[]
for k in range(K):
    md3 = Model(name='scenario_analysis')
    x = {(i): md3.continuous_var(name='x_{0}'.format(i)) for i in range(num_prod)}

    for i in range(m):
        md3.add_constraint(md3.sum(x[j]*prod_rate[i][j] for j in range(num_prod)) >= demands[i]+h_scenario[k][i])
    md3.add_constraint(md3.sum(x[j] for j in range(num_prod))<=capacity)
    obj3=md3.minimize(md3.sum(x[i]*c[i] for i in range(num_prod)))
    outputs=str(md3.solve()).split("\n")
    obj_v=float(outputs[1].split(": ")[1])
    scenario_perf.append(obj_v)
    scenario_x.append(outputs[2:-1])
max(scenario_perf),scenario_x[np.argmax(scenario_perf)]

(145.258, ['x_0=39.994', 'x_1=21.757'])

### 4. chance-constrainted

In [10]:
mu_1,sigma_1=0,16
mu_2,sigma_2=0,9
mus=[mu_1,mu_2]
sigmas=[sigma_1,sigma_2]
epsilon=0.05

In [11]:
md4 = Model(name='chance_constrainted')
x = {(i): md4.continuous_var(name='x_{0}'.format(i)) for i in range(num_prod)}

md4.add_constraint(md4.sum(x[j] for j in range(num_prod))<=capacity)

for i in range(m):
    md4.add_constraint(md4.sum(x[j]*prod_rate[i][j] for j in range(num_prod)) >= demands[i]+mus[i]+norm.ppf(1-epsilon)*sigmas[i])

obj4=md4.minimize(md4.sum(x[i]*c[i] for i in range(num_prod)))
md4.print_information()
md4.solve().display()

Model: chance_constrainted
 - number of variables: 2
   - binary=0, integer=0, continuous=2
 - number of constraints: 3
   - linear=3
 - parameters: defaults
solution for: chance_constrainted
objective: 140.481
x_0 = 37.322
x_1 = 21.945


### 5. model with recourse

In [12]:
c_rec=[7,12]

In [13]:
md5 = Model(name='recourse_model')
x = {(i): md5.continuous_var(name='x_{0}'.format(i),lb=0) for i in range(num_prod)}
y = {(k,i): md5.continuous_var(name='y_{0}_{1}'.format(k,i),lb=0) for i in range(num_prod) for k in range(K)}

md5.add_constraint(md5.sum(x[j] for j in range(num_prod))<=capacity)

for k in range(K):
    for i in range(m):
        md5.add_constraint(md5.sum(x[j]*prod_rate[i][j] for j in range(num_prod))\
                           +y[k,i] >= demands[i]+h_scenario[k][i])

obj5=md5.minimize(md5.sum(x[i]*c[i] for i in range(num_prod))\
                  +md5.sum(md5.sum(y[k,i]*c_rec[i] for i in range(num_prod)) for k in range(K))/K)
md5.print_information()
md5.solve().display()

Model: recourse_model
 - number of variables: 1002
   - binary=0, integer=0, continuous=1002
 - number of constraints: 1001
   - linear=1001
 - parameters: defaults
solution for: recourse_model
objective: 144.296
x_0 = 37.229
x_1 = 22.054
y_0_0 = 3.677
y_44_0 = 4.028
y_89_0 = 2.127
y_120_0 = 2.599
y_148_0 = 9.144
y_149_0 = 3.186
y_180_0 = 11.726
y_181_0 = 3.747
y_204_0 = 7.207
y_227_0 = 1.796
y_255_0 = 7.561
y_263_0 = 0.222
y_320_0 = 30.708
y_385_0 = 8.960
y_416_0 = 2.340
y_425_0 = 0.337
y_475_0 = 6.601
y_110_1 = 7.062
y_172_1 = 2.872
y_181_1 = 7.404
y_243_1 = 3.009
y_253_1 = 0.057
y_262_1 = 5.331
y_263_1 = 0.020
y_269_1 = 2.679
y_277_1 = 4.514
y_293_1 = 2.752
y_321_1 = 5.351
y_330_1 = 6.633
y_360_1 = 3.500
y_367_1 = 6.289
y_381_1 = 0.506
y_429_1 = 8.776
y_435_1 = 0.153
y_459_1 = 5.356
y_472_1 = 2.939
y_479_1 = 16.207


### Evaluate the performances

In [15]:
Decisions={}
Decisions["deterministic"]={}
Decisions["deterministic"]["decisions"]=[36.500,17.833]
Decisions["deterministic"]["obj_v"]=sum(np.array(Decisions["deterministic"]["decisions"])*np.array(c))

Decisions["worst_case"]={}
Decisions["worst_case"]["decisions"]=[37.663,23.649]
Decisions["worst_case"]["obj_v"]=sum(np.array(Decisions["worst_case"]["decisions"])*np.array(c))

Decisions["scenario_analysis"]={}
Decisions["scenario_analysis"]["decisions"]=[39.994,21.757]
Decisions["scenario_analysis"]["obj_v"]=sum(np.array(Decisions["scenario_analysis"]["decisions"])*np.array(c))

Decisions["chance_constrainted"]={}
Decisions["chance_constrainted"]["decisions"]=[37.322,21.945]
Decisions["chance_constrainted"]["obj_v"]=sum(np.array(Decisions["chance_constrainted"]["decisions"])*np.array(c))

Decisions["recourse_model"]={}
Decisions["recourse_model"]["decisions"]=[37.229,22.054]
Decisions["recourse_model"]["obj_v"]=sum(np.array(Decisions["recourse_model"]["decisions"])*np.array(c))
for key in Decisions.keys():
    Decisions[key]["feasibility"]=0

In [16]:
for k in range(5000):
    h_1,h_2=np.random.normal(loc=0,scale=16),np.random.normal(loc=0,scale=9)
    for key in Decisions.keys():
        if sum([Decisions[key]['decisions'][i]*prod_rate[0][i] for i in range(num_prod)])>=180+h_1 and \
        sum([Decisions[key]['decisions'][i]*prod_rate[1][i] for i in range(num_prod)])>=163+h_2:
            Decisions[key]["feasibility"]+=1

In [17]:
for key in Decisions.keys():
    Decisions[key]["feasibility"]/=5000

In [18]:
Decisions

{'deterministic': {'decisions': [36.5, 17.833],
  'obj_v': 126.499,
  'feasibility': 0.2514},
 'worst_case': {'decisions': [37.663, 23.649],
  'obj_v': 146.273,
  'feasibility': 0.981},
 'scenario_analysis': {'decisions': [39.994, 21.757],
  'obj_v': 145.25900000000001,
  'feasibility': 0.968},
 'chance_constrainted': {'decisions': [37.322, 21.945],
  'obj_v': 140.479,
  'feasibility': 0.9076},
 'recourse_model': {'decisions': [37.229, 22.054],
  'obj_v': 140.62,
  'feasibility': 0.91}}

As we can see, the model chance constrainted has a small enough objective function value and it has good feasiblity